# Process annotations

# Post-Processing with LLM (Qwen3-0.6B)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")

# Test the model
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

In [ ]:
def fix_ocr_qwen(ocr_text, context=""):
    prompt = f"""Bạn là trợ lý chỉnh sửa văn bản manga.
Nhiệm vụ: Sửa lỗi OCR tiếng Nhật dựa trên ngữ cảnh. Chỉ in ra kết quả.
Context: {context}
Input: {ocr_text}
Output:"""
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.1,  # Để thấp để model ít bịa đặt
        top_p=0.9
    )
    
    generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
    return generated_text.split("Output:")[-1].strip()

In [ ]:
# Test với data của bạn
print(fix_ocr_qwen("いくつ", context="Đi học thôi")) 
# Hy vọng nó sửa thành "行くの" hoặc tương tự